# Chapter 5: The Dirty COW vulnerability in Rust

## Introduction
[**The Dirty COW vulnerability**](https://dirtycow.ninja/) represents a fascinating instance of a [**race condition**](https://en.wikipedia.org/wiki/Race_condition) [**vulnerability**](https://en.wikipedia.org/wiki/Vulnerability_(computing)) within the [**Linux kernel**](https://en.wikipedia.org/wiki/Linux_kernel). This flaw has been present in the kernel since **September 2007** but only came to light when [**it was discovered**](https://access.redhat.com/security/cve/CVE-2016-5195) and [**patched by the lord himself**](https://git.kernel.org/pub/scm/linux/kernel/git/torvalds/linux.git/commit/?id=19be0eaffa3ac7d8eb6784ad9bdbc7d67ed8e619) in **October 2016**. What sets this vulnerability apart is its wide-reaching impact, affecting all Linux-based operating systems, including the popular Android platform. The gravity of the situation lies in the potential consequences, as attackers exploiting this vulnerability can acquire root privileges, granting them god-tier control over the compromised system.

At its core, the vulnerability is embedded in the [**copy-on-write**](https://en.wikipedia.org/wiki/Copy-on-write) mechanism within the Linux kernel's code. The exploit allows attackers to manipulate protected files, even those designated as exclusively readable by them. This chapter delves into the complexities of the attack, dissecting its mechanisms and demonstrating how bad actors can leverage it to modify critical system files. A noteworthy example is the manipulation of the **`/etc/password`** file, showcasing how attackers can exploit the Dirty COW vulnerability to elevate their privileges to the root level, effectively taking over the entire system.

To fully comprehend the Dirty COW race condition vulnerability, it is crucial to explore its historical context. This flaw went undetected for almost a decade, highlighting the sneaky nature of certain security threats. The vulnerability was brought to light through meticulous research and analysis, emphasizing the perpetual need for careful investigation in the world of cybersecurity. Moreover, its discovery underscores the challenges inherent in maintaining the security of open-source systems, where complex codebases can port vulnerabilities over extended periods.

In terms of practical implications, the Dirty COW vulnerability has prompted widespread concern within the cybersecurity community. Security experts and Linux system administrators must remain careful, promptly patching affected systems and implementing robust security measures. The incident also serves as a stark reminder of the ever-evolving nature of cyber threats and the necessity for proactive defense mechanisms.

## Table of Contents

* [**Memory Mapping**](#1.-Memory-Mapping)
    * [**Applications of Memory Mapping**](#1.1-Applications-of-Memory-Mapping)
       * [**File I/O Operations with Memory Mapping**](#1.1.1-File-I/O-Operations-with-Memory-Mapping)
       * [**Memory-Mapped Database**](#1.1.2-Memory-Mapped-Database)
       * [**Memory-Mapped Networking**](#1.1.3-Memory-Mapped-Networking)

## 1. Memory Mapping

Kicking off the journey to comprehend the complexities of the **Dirty COW** vulnerability necessitates a solid grasp of the foundational concept of memory mapping through the use of the [**`libc::mmap`**](https://docs.rs/libc/latest/libc/fn.mmap.html) method. Within the Unix operating system, [**`mmap`**](https://man7.org/linux/man-pages/man2/mmap.2.html) empowers the seamless integration of files or devices into a process's memory space. This mechanism plays a crucial role in shaping how data is accessed and manipulated within a computer system.

By default, **`mmap`** employs [**file-backed mapping**](https://en.wikipedia.org/wiki/Mmap#File-backed_and_anonymous), establishing a [**symbiotic relationship**](https://en.wikipedia.org/wiki/Symbiosis) between an allocated portion of a process's virtual memory and corresponding files. When information is read from the mapped area, it dynamically translates into the retrieval of data from the associated file. This natural connection between memory and file operations forms the backbone of **`mmap`**'s functionality.

To shed light on this process, let's turn our attention to the following code snippet. This code snippet encapsulates the essence of memory mapping, showcasing how **`mmap`** is employed to create a link between a file and a process's memory space. This practical example offers valuable insights into the mechanics of memory mapping, serving as a guide for understanding the subsequent exploration of the Dirty COW vulnerability.

In [2]:
:dep libc = { version = "0.2.151" }

In [4]:
use libc::{
    MAP_FAILED, MAP_SHARED, PROT_READ, PROT_WRITE,
};
use std::fs::OpenOptions;
use std::io;
use std::os::fd::AsRawFd;
use std::ptr;
use std::slice;

fn main() -> io::Result<()> {
    let mut file_stat = unsafe { std::mem::zeroed() };
    let mut file_content: [u8; 10] = [0; 10];
    let new_data = "\nUpdated Data\n";
    let file_path = "file.txt";

    let file = OpenOptions::new().read(true).write(true).open(&file_path)?;

    unsafe {
        libc::fstat(file.as_raw_fd(), &mut file_stat);

        let mapped_memory = libc::mmap(
            ptr::null_mut(),
            file_stat.st_size as usize,
            PROT_READ + PROT_WRITE,
            MAP_SHARED,
            file.as_raw_fd(),
            0,
        );

        if mapped_memory == MAP_FAILED {
            return Err(io::Error::last_os_error());
        }

        let mapped_slice = slice::from_raw_parts(mapped_memory as *const u8, 10);
        file_content.copy_from_slice(mapped_slice);
        println!("Read: {}", String::from_utf8_lossy(&file_content));

        let new_data_bytes = new_data.as_bytes();
        let write_offset = 5;
        if file_stat.st_size as usize >= write_offset + new_data_bytes.len() {
            ptr::copy_nonoverlapping(
                new_data_bytes.as_ptr(),
                (mapped_memory as *mut u8).wrapping_add(write_offset),
                new_data_bytes.len(),
            );
            println!("Write successful at offset {} with data: {}", write_offset, new_data);
        } else {
            eprintln!("Write offset exceeds file size. Update not performed.");
        }

        libc::munmap(mapped_memory, file_stat.st_size as usize);
    }

    Ok(())
}

main()

Read: Hello Rust
Write successful at offset 5 with data: 
Updated Data



Ok(())

```sh
+---------------------------+
|                           |
|   Open file.txt           |
|                           |
+-------------+-------------+
              |
              v
+-------------+-------------+
|                           |
|   Get file information    |
|   using fstat()           |
|                           |
+-------------+-------------+
              |
              v
+-------------+-------------+
|                           |
|   Map file into memory    |
|   with read and write     |
|   permissions             |
|                           |
+-------------+-------------+
              |
              v
+-------------+-------------+
|                           |
|   Check if mapping        |
|   is successful           |
|                           |
+-------------+-------------+
              |
              v
+-------------+-------------+
|                           |
|   Read 10 bytes from the  |
|   mapped memory           |
|                           |
+-------------+-------------+
              |
              v
+-------------+-------------+
|                           |
|   Copy read data to       |
|   file_content array      |
|                           |
+-------------+-------------+
              |
              v
+-------------+-------------+
|                           |
|   Write new data to       |
|   mapped memory at        |
|   offset 5                |
|                           |
+-------------+-------------+
              |
              v
+-------------+-------------+
|                           |
|   Unmap the memory        |
|                           |
+-------------+-------------+
```

In this code snippet, we make use of the Rust standard library and incorporate certain unsafe bindings to work with the **mmap system call** and its associated operations. The utilization of the **unsafe** block is essential for managing **low-level** operations, particularly the direct copying of memory.

The program opens a file named **file.txt**, maps its entire content into memory, performs read and write operations on the mapped memory, and concludes with necessary cleanup operations. A detailed analysis of this code is essential for gaining insights into the complexities of **`mmap`** usage.

This code snippet illustrates the **`mmap`** system call's functionality in generating a mapped memory region. Key parameters, including the **starting address**, **size**, and **accessibility**, are explicitly defined to ensure alignment with file operations. Furthermore, common memory operations such as **reading** and **writing** to the mapped memory are executed through Rust's standard library functions, ensuring both type safety and effective memory management.

Once a file is successfully mapped to memory, subsequent operations become streamlined. For instance, the **`memcpy`** method invocation to read a specific number of bytes from the file, utilizing the advantages of memory access. Similarly, another **`memcpy`** method invocation to write a designated string to the file, thereby modifying its content. These operations exemplify the efficiency and convenience that memory mapping offers in handling file data.

Comprehending **mmap** is crucial, particularly in the context of Dirty COW, where the exploitation depends on manipulating memory mappings to obtain unauthorized access. The Rust programming language, famous for its emphasis on safety and performance, serves as a proficient platform for navigating the complexities of low-level memory interactions while maintaining the integrity of the codebase.

### 1.1 Applications of Memory Mapping

Memory mapping in Rust is like having a super powerful tool in your programming toolkit. It's like a Swiss Army knife for dealing with various real-world applications. Rust makes using memory mapping super easy, giving us a powerful way to solve lots of different problems. Now, let's take a closer look at five specific applications where memory mapping in Rust shines, showing off how flexible and useful it can be.

#### 1.1.1 File I/O Operations with Memory Mapping

Memory mapping is a game-changer in the context of file I/O operations within Rust. The following code snippet presents a sophisticated approach to file handling, demonstrating the coordination between Rust's robust capabilities and memory mapping. Opening a file, setting its size, and mapping it into mutable memory become elegant operations thanks to the [**`memmap`**](https://docs.rs/memmap) crate, a Rust library designed for memory mapping. This example goes beyond mere file manipulation; it transforms the process into a seamless operation where direct in-memory manipulations can occur. The elimination of explicit read-and-write operations enhances both the clarity and performance of the code, particularly beneficial when dealing with large files requiring efficient processing and modification.

```rust
use std::fs::OpenOptions;
use std::io::{Read, Write};

fn main() -> std::io::Result<()> {
    let file = OpenOptions::new()
        .read(true)
        .write(true)
        .create(true)
        .open("example.txt")?;

    let size = 1024;

    file.set_len(size as u64)?;

    // Map the file into memory
    let mut content = unsafe {
        memmap::MmapMut::map_mut(&file)?
    };

    // Perform in-memory operations
    content[0] = b'A';

    // Changes are automatically reflected in the file

    Ok(())
}
```

```sh
+---------------------+
|    Open File        |
|---------------------|
|  - Readable         |
|  - Writable         |
|  - Create if absent |
+---------------------+
                |
                v
+---------------------+            +----------------------+
|   Set File Size     |            |                      |
|---------------------|            |                      |
|  - Specify Size     |            |    Content in        |
+---------------------+            |                      |
                |                  |       Memory         |
                v                  v                      |
+---------------------+  File Size +----------------------+
|   Map into Memory   |----------->|                      |
|---------------------|            |                      |
|  - Use `memmap`     |            |                      |
+---------------------+            |                      |
                |                  |                      |
                v                  |                      |
+---------------------+            |                      |
|  In-Memory Changes  |            |                      |
|---------------------|            |                      |
|  - Directly modify  |            |                      |
|    in memory        |            |                      |
+---------------------+            |                      |
                |                  |                      |
                v                  |                      |
+---------------------+            |                      |
| Auto Reflection in  |<----------+|                      |
|   the File          |            |                      |
|---------------------|            |                      |
|  - Changes reflect  |            |                      |
|    in the file      |            |                      |
+---------------------+            |                      |
                |                  |                      |
                v                  |                      |
+---------------------+            |                      |
|       Cleanup       |            |                      |
|---------------------|            |                      |
|  - Unmap the memory |            |                      |
|  - Close the file   |            |                      |
+---------------------+            |                      |
                                   |                      |
                                   +----------------------+

```

This code performs file I/O operations with memory mapping. It first opens a file named "example.txt" with read and write permissions, creating the file if it doesn't exist. It then sets the size of the file to 1024 bytes. Using the `memmap` crate, the code maps the entire file into mutable memory, creating a direct link between the program and the file. Subsequently, it performs in-memory operations by modifying the first byte of the content to the ASCII value of 'A'. Notably, any changes made in memory are automatically reflected in the file.

In [2]:
:dep memmap = { version = "0.7.0" }

In [3]:
use std::fs::OpenOptions;
use std::io::{Read, Write};

fn main() -> std::io::Result<()> {
    let file = OpenOptions::new()
        .read(true)
        .write(true)
        .create(true)
        .open("example.txt")?;

    let size = 1024;

    file.set_len(size as u64)?;

    // Map the file into memory
    let mut content = unsafe {
        memmap::MmapMut::map_mut(&file)?
    };

    // Perform in-memory operations
    content[0] = b'A';

    // Changes are automatically reflected in the file
    Ok(())
}

main()

Ok(())

#### 1.1.2 Memory-Mapped Database

The integration of memory mapping in database management showcases Rust's prowess in handling high-performance scenarios. In the presented snippet, the [**`sled`**](https://github.com/spacejam/sled) crate is employed to create an embedded key-value store with memory mapping. This example showcases the practical application of memory mapping in databases, where blazingly fast and efficient access to data is crucial. The `sled` crate, leveraging memory mapping, provides an interface for key-value pair operations, ensuring the persistence and retrieval of data with optimal performance characteristics. The combination of Rust's memory safety guarantees and the efficiency of memory mapping positions the language as a compelling choice for developing performant and reliable database systems.

```rust
use sled::Db;

fn main() {
    let db = Db::start_default("my_db").unwrap();

    db.insert(b"key1", b"value1").unwrap();
    db.insert(b"key2", b"value2").unwrap();

    if let Some(value) = db.get(b"key1").unwrap() {
        println!("Value for key1: {:?}", value);
    }
}
```

```sh
+--------------------------+
|  Open or Create Database |
|--------------------------|
|  - Initialize Database   |
+------------------------- +
                |
                v
+--------------------------------+      +------------------------+
|    Insert Key-Value Pairs      |      |                        |
|--------------------------------|      |      Content in        |
| - Key: "key1", Value: "value1" +----->|        Memory          |
| - Key: "key2", Value: "value2" |      |                        |
+--------------------------------+      |                        |
                |                       +------------------------+
                |                                    |
                v                                    v
+-------------------------+             +------------------------+
|   Read Values from      |<-----------+|                        |
|     the Database        |             |                        |
|-------------------------|             |                        |
|  - Read Value for "key1"|             |                        |
+-------------------------+             |                        |
                |                       |                        |
                v                       |                        |
+-------------------------+             |                        |
|        End              |             |                        |
|-------------------------|             |                        |
|  - Database Cleanup     |             |                        |
+-------------------------+             |                        |
                                        +------------------------+
```

In this code snippet, the `sled` crate is employed to showcase the seamless management of a memory-mapped database. The `main` function initiates a new database, "my_db", demonstrating the simplicity of database initialization with `sled`. Two key-value pairs, associating "key1" with "value1" and "key2" with "value2," are efficiently inserted into the database using the `insert` method. The subsequent operation involves reading the value associated with "key1" from the database using the `get` method. The use of memory mapping by the `sled` crate ensures quick and direct access to the data, enhancing performance and reliability. The code encapsulates the essence of Rust's expressiveness in systems programming, emphasizing its proficiency in managing high-performance memory-mapped databases.

In [6]:
:dep sled = { version = "0.34.7" }

In [11]:
use sled::Db;

let db: Db = sled::open("my_db").unwrap();

db.insert(b"key1", b"value1").unwrap();
db.insert(b"key2", b"value2").unwrap();

if let Some(value) = db.get(b"key1").unwrap() {
    let readable_value = String::from_utf8_lossy(&value);
    println!("Value for key1: {}", readable_value);
}

Value for key1: value1


()

#### 1.1.3 Memory-Mapped Networking

Memory mapping proves advantageous in enhancing network programming in Rust, particularly with asynchronous I/O operations. The example utilizes the [**`mio`**](https://docs.rs/mio) crate for building a simple asynchronous TCP server. The code demonstrates how memory-mapped buffers can be employed to handle data on existing connections efficiently. This example illustrates the coordination between memory mapping and asynchronous I/O, showcasing its potential to streamline networking applications in Rust.

```rust
use mio::net::{TcpListener, TcpStream};
use mio::{Events, Interest, Poll, Token};
use std::io::Read;
use std::net::SocketAddr;

fn main() {
    let addr: SocketAddr = "127.0.0.1:8080".parse().unwrap();
    let mut listener = TcpListener::bind(addr).unwrap();

    let mut poll = Poll::new().unwrap();
    let mut events = Events::with_capacity(1024);

    poll.registry()
        .register(&mut listener, Token(0), Interest::READABLE)
        .unwrap();
    
    let mut connections = Vec::new();

    loop {
        poll.poll(&mut events, None).unwrap();

        for event in &events {
            if event.token() == Token(0) && event.is_readable() {
                // Accept incoming connection and create a new connection object
                let (stream, _) = listener.accept().unwrap();
                connections.push(stream);
                println!("New connection accepted!");
            } else {
                // Handle data on existing connections using memory-mapped buffers
                let mut buffer = [0; 1024];
                let stream_index = event.token().0 as usize - 1;
                let mut stream = &connections[stream_index];
                match stream.read(&mut buffer) {
                    Ok(0) => {
                        println!("Connection closed by client");
                    }
                    Ok(bytes_read) => {
                        println!("Received {} bytes of data: {:?}", bytes_read, &buffer[..bytes_read]);
                        // Process the received data
                        // ...
                    }
                    Err(err) => {
                        println!("Error reading from the connection: {:?}", err);
                    }
                }
            }
        }
    }
}

main()
```

```sh
+------------------------+                +------------------------+
|                        |                |                        |
|       Rust Program     |                |      Client using      |
|                        |                |        netcat          |
+------------------------+                +------------------------+
             |                                       |
             |                                       |
             v                                       v
+------------------------+                +------------------------+
|                        |                |                        |
|    TCP Listener:       |                |   Connected Client:    |
|   127.0.0.1:8080       |<---------------+   Connected to         |
|                        |                |   127.0.0.1:8080       |
|                        |                |                        |
+------------------------+                +------------------------+
             |                                       
             |                                       
             v                                       
+------------------------+                           
|                        |                           
|   Mio Poll and Events  |                           
|                        |                           
+-----------+------------+                           
            |                                        
            |                                        
            v                                        
+-----------+------------+                           
|                        |                           
|   Register TCP         |                           
|   Listener with Mio    |                           
|                        |                           
|                        |                           
+-----------+------------+                           
            |                                       
            |                                        
            v                                        
+------------------------+
|                        |
|   Event Processing     |
|                        |
|   - Accept new         |
|     connections        |
|   - Handle data on     |
|     existing           |
|     connections        |
+------------------------+
```

This program establishes a simple TCP server using `mio`, designed for asynchronous I/O. The server binds to the address `127.0.0.1:8080`, creating a TCP listener to accept incoming connections. The program utilizes the `mio` event-driven framework, employing a non-blocking approach to handle multiple I/O operations concurrently. The `Poll` structure is employed to monitor events, and an event capacity of 1024 is specified. A `Token` with a value of 0 is registered with the listener for readability. The program maintains a vector, `connections`, to store active TCP streams established with clients.

The main loop continuously polls for events, responding to incoming connections and managing data on existing connections. When an event indicates readability and is associated with the registered token (0), the program accepts the incoming connection, creating a new TCP stream and adding it to the `connections` vector. On events associated with other tokens, the program reads data from the corresponding connection into a 1024-byte buffer using asynchronous I/O operations. If the read operation returns 0 bytes, indicating the client closed the connection, the associated stream is removed from the `connections` vector. Otherwise, the program processes the received data, allowing for further application-specific handling.

This example showcases the efficient handling of data on existing connections using memory-mapped buffers, emphasizing the utility of memory mapping in networking scenarios. The combination of Rust's safety features and memory mapping's efficiency positions Rust as a robust choice for building high-performance networking applications.

In [28]:
:dep mio = { version = "0.8.10", features=["os-poll", "net"] }

In [ ]:
use mio::net::{TcpListener, TcpStream};
use mio::{Events, Interest, Poll, Token};
use std::io::Read;
use std::net::SocketAddr;

fn main() {
    let addr: SocketAddr = "127.0.0.1:8080".parse().unwrap();
    let mut listener = TcpListener::bind(addr).unwrap();

    let mut poll = Poll::new().unwrap();
    let mut events = Events::with_capacity(1024);

    poll.registry()
        .register(&mut listener, Token(0), Interest::READABLE)
        .unwrap();
    
    let mut connections = Vec::new();

    loop {
        poll.poll(&mut events, None).unwrap();

        for event in &events {
            if event.token() == Token(0) && event.is_readable() {
                // Accept incoming connection and create a new connection object
                let (stream, _) = listener.accept().unwrap();
                connections.push(stream);
                println!("New connection accepted!");
            } else {
                // Handle data on existing connections using memory-mapped buffers
                let mut buffer = [0; 1024];
                let stream_index = event.token().0 as usize - 1;
                let mut stream = &connections[stream_index];
                match stream.read(&mut buffer) {
                    Ok(0) => {
                        println!("Connection closed by client");
                    }
                    Ok(bytes_read) => {
                        println!("Received {} bytes of data: {:?}", bytes_read, &buffer[..bytes_read]);
                        // Process the received data
                        // ...
                    }
                    Err(err) => {
                        println!("Error reading from the connection: {:?}", err);
                    }
                }
            }
        }
    }
}

main()

New connection accepted!


---
---